In [143]:
from rdflib import Graph, Namespace, RDF

# File path to the ontology
ontology_file = 'Latest Ontology.ttl'

# Create a graph object
graph = Graph()

# Load the RDF data from the Turtle file
graph.parse(ontology_file, format='ttl')

# Define the namespace
ex = Namespace('http://example.org#')



In [128]:
graph.__len__()

448

In [129]:
q = """ PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>

SELECT ?course
WHERE {
  ?course rdf:type ex:Course.
}
"""

results = graph.query(q)
print("---------------------------------------")
print("|           Courses in KB:            |")
print("---------------------------------------")
for row in results:
    print(str(row.course).split("/")[-1])


---------------------------------------
|           Courses in KB:            |
---------------------------------------
NLP
IS_Audit_And_Control
DesignDefect


In [186]:
q = """ PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>

SELECT ?teacher
WHERE {
  ?teacher rdf:type ex:Teacher.
}
"""

results = graph.query(q)
print("---------------------------------------")
print("|           Teachers in KB:            |")
print("---------------------------------------")
for row in results:
    print(str(row.teacher).split("/")[-1])


---------------------------------------
|           Teachers in KB:            |
---------------------------------------
Mr_Kashif
Dr_Sarim
Mr_Syed_Yousuf
Mr_Ali_Shan


Teacher Matching the Skills Requirements

In [131]:
# course = "DesignDefect"
course = "IS_Audit_And_Control"
# course = "NLP"

q = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?skill) AS ?matchedSkillCount) 
WHERE {{
    ?teacher ex:hasSkill ?skill.
    ex:{course} ex:requiresSkill ?skill.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedSkillCount)"""

results = graph.query(q)

print("Following teacher has some skills relevant to course: ",course)
for tuple in results:
    print(str(tuple.teacher).split("/")[-1],end="\t")
    print(tuple.matchedSkillCount)


Following teacher has some skills relevant to course:  IS_Audit_And_Control
Mr_Kashif	13
Mr_Syed_Yousuf	1
Mr_Ali_Shan	1


Job Experience Requirements

In [132]:
q = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?role) AS ?matchedCount)  
WHERE {{
    ?teacher ex:hasExperience ?exp.
    ?exp ex:hasRole ?role.
    ?role ex:roleRelevantTo ex:{course}.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedCount)
"""

results = graph.query(q)

print("Following teacher has some Job Experience relevant to course: ",course)
for tuple in results:
    print(str(tuple.teacher).split("/")[-1],end="\t")
    print(tuple.matchedCount)


Following teacher has some Job Experience relevant to course:  IS_Audit_And_Control
Mr_Kashif	2
Mr_Syed_Yousuf	2


Weighted Job Expereience Requirements

In [137]:
q = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (SUM(?years) AS ?totalYears) (COUNT(?role) AS ?matchedCount)  
WHERE {{
    ?teacher ex:hasExperience ?exp.
    ?exp ex:hasRole ?role.
    ?role ex:roleRelevantTo ex:{course}.
    ?exp ex:yearsOfExperience ?years.
}}
GROUP BY ?teacher
ORDER BY DESC(?totalYears)

"""

results = graph.query(q)

print("Following teacher has some Job Experience (Weighted With Years) In relevant to course: ",course)
for tuple in results:
    print(str(tuple.teacher).split("/")[-1],end="\t")
    print(tuple.totalYears)


Following teacher has some Job Experience (Weighted With Years) In relevant to course:  IS_Audit_And_Control
Mr_Kashif	19
Mr_Syed_Yousuf	6


Research Work Relevant To Course

In [139]:
# Research Work Relevant To Course
q = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?research) AS ?matchedCount)  
WHERE {{
    ?teacher ex:doneResearch ?research.
    ?research ex:researchRelatedTo ex:{course}.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedCount)
"""

results = graph.query(q)

print("Following teacher has some Research Experience relevant to course: ",course)
for tuple in results:
    print(str(tuple.teacher).split("/")[-1],end="\t")
    print(tuple.matchedCount)


Following teacher has some Research Experience relevant to course:  IS_Audit_And_Control
Mr_Kashif	4


Course and Teachers in How many Similar Domain

In [140]:
# Research Work Relevant To Course
q = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?domain) AS ?matchedCount)  
WHERE {{
    ?teacher ex:relatedToDomain ?domain.
    ex:{course} ex:belongsToDomain ?domain.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedCount)

"""

results = graph.query(q)

print("Course and Teachers in How many Similar Domain ",course)
for tuple in results:
    print(str(tuple.teacher).split("/")[-1],end="\t")
    print(tuple.matchedCount)


Course and Teachers in How many Similar Domain  IS_Audit_And_Control
Mr_Kashif	1
Dr_Sarim	1
Mr_Syed_Yousuf	1
Mr_Ali_Shan	1


Number of Matching Degrees Teacher has relevant to the Course

In [144]:
q = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?degree) AS ?matchedCount)  
WHERE {{
    ?teacher ex:hasDegree ?degree.
    ex:{course} ex:requiresDegree ?degree.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedCount)
"""

results = graph.query(q)

print("Number of Matching Degrees Teacher has relevant to the Course: ",course)
for tuple in results:
    print(str(tuple.teacher).split("/")[-1],end="\t")
    print(tuple.matchedCount)


Number of Matching Degrees Teacher has relevant to the Course:  IS_Audit_And_Control
Mr_Kashif	2
Mr_Ali_Shan	1


All Queries

In [239]:

course = "IS_Audit_And_Control"

q1 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?skill) AS ?matchedSkillCount) 
WHERE {{
    ?teacher ex:hasSkill ?skill.
    ex:{course} ex:requiresSkill ?skill.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedSkillCount)"""


q2 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?role) AS ?matchedCount)  
WHERE {{
    ?teacher ex:hasExperience ?exp.
    ?exp ex:hasRole ?role.
    ?role ex:roleRelevantTo ex:{course}.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedCount)
"""


q3 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (SUM(?years) AS ?totalYears) (COUNT(?role) AS ?matchedCount)  
WHERE {{
    ?teacher ex:hasExperience ?exp.
    ?exp ex:hasRole ?role.
    ?role ex:roleRelevantTo ex:{course}.
    ?exp ex:yearsOfExperience ?years.
}}
GROUP BY ?teacher
ORDER BY DESC(?totalYears)

"""

q4 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?research) AS ?matchedCount)  
WHERE {{
    ?teacher ex:doneResearch ?research.
    ?research ex:researchRelatedTo ex:{course}.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedCount)
"""

q5 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?domain) AS ?matchedCount)  
WHERE {{
    ?teacher ex:relatedToDomain ?domain.
    ex:{course} ex:belongsToDomain ?domain.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedCount)

"""

q6 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?degree) AS ?matchedCount)  
WHERE {{
    ?teacher ex:hasDegree ?degree.
    ex:{course} ex:requiresDegree ?degree.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedCount)
"""


Data Frame for Analysis

In [240]:
import pandas as pd


scores = {}

# Query 1: Matching Skills
results = graph.query(q1)
for result in results:
    teacher = str(result.teacher).split("/")[-1]
    matched_skills = int(result.matchedSkillCount)
    if teacher not in scores:
        scores[teacher] = {"matchedSkills": 0, "jobExperience": 0, "weightedExperience": 0, "researchExperience": 0, "domainMatching": 0, "degreeMatching": 0}
    scores[teacher]["matchedSkills"] = matched_skills

# Query 2: Job Experience for Diversity
results = graph.query(q2)
for result in results:
    teacher = str(result.teacher).split("/")[-1]
    matched_exp = int(result.matchedCount)
    if teacher not in scores:
        scores[teacher] = {"matchedSkills": 0, "jobExperience": 0, "weightedExperience": 0, "researchExperience": 0, "domainMatching": 0, "degreeMatching": 0}
    scores[teacher]["jobExperience"] = matched_exp

# Query 3: Weighted Experience Overall Aggregated
results = graph.query(q3)
for result in results:
    teacher = str(result.teacher).split("/")[-1]
    total_years = int(result.totalYears)
    if teacher not in scores:
        scores[teacher] = {"matchedSkills": 0, "jobExperience": 0, "weightedExperience": 0, "researchExperience": 0, "domainMatching": 0, "degreeMatching": 0}
    scores[teacher]["weightedExperience"] = total_years

# Query 4: Research Experience
results = graph.query(q4)
for result in results:
    teacher = str(result.teacher).split("/")[-1]
    matched_research = int(result.matchedCount)
    if teacher not in scores:
        scores[teacher] = {"matchedSkills": 0, "jobExperience": 0, "weightedExperience": 0, "researchExperience": 0, "domainMatching": 0, "degreeMatching": 0}
    scores[teacher]["researchExperience"] = matched_research

# Query 5: Domain Matching
results = graph.query(q5) 
for result in results:
    teacher = str(result.teacher).split("/")[-1]
    matched_domains = int(result.matchedCount)
    if teacher not in scores:
        scores[teacher] = {"matchedSkills": 0, "jobExperience": 0, "weightedExperience": 0, "researchExperience": 0, "domainMatching": 0, "degreeMatching": 0}
    scores[teacher]["domainMatching"] = matched_domains

# Query 6: Degree Matching
results = graph.query(q6)
for result in results:
    teacher = str(result.teacher).split("/")[-1]
    matched_degrees = int(result.matchedCount)
    if teacher not in scores:
        scores[teacher] = {"matchedSkills": 0, "jobExperience": 0, "weightedExperience": 0, "researchExperience": 0, "domainMatching": 0, "degreeMatching": 0}
    scores[teacher]["degreeMatching"] = matched_degrees


df = pd.DataFrame.from_dict(scores, orient="index")
df.index.name = "Teacher"
df.reset_index(inplace=True)
df['Score'] = df["matchedSkills"] + df["jobExperience"] + df["weightedExperience"] + df["researchExperience"] + df["domainMatching"] + df["degreeMatching"]
best_teacher = df[ df["Score"] == df["Score"].max()]["Teacher"].values[0]


In [241]:
print(best_teacher)
df

Mr_Kashif


,Teacher,matchedSkills,jobExperience,weightedExperience,researchExperience,domainMatching,degreeMatching,Score
0,Mr_Kashif,13,2,19,4,1,2,41
1,Mr_Syed_Yousuf,1,2,6,0,1,0,10
2,Mr_Ali_Shan,1,0,0,0,1,1,3
3,Dr_Sarim,0,0,0,0,1,0,1


Get Matched Entities

In [242]:
q1 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?skill
WHERE {{
    ex:{best_teacher} ex:hasSkill ?skill.
    ex:{course} ex:requiresSkill ?skill.
}} """


q2 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?exp ?role ?years ?organization
WHERE {{
    ex:{best_teacher} ex:hasExperience ?exp.
    ?exp ex:hasRole ?role.
    ?role ex:roleRelevantTo ex:{course}.
    ?exp ex:yearsOfExperience ?years. 
    ?exp ex:organization ?organization.
}}"""



q3 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?research
WHERE {{
    ex:{best_teacher} ex:doneResearch ?research.
    ?research ex:researchRelatedTo ex:{course}.
}}
"""

q4 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?domain
WHERE {{
    ex:{best_teacher} ex:relatedToDomain ?domain.
    ex:{course} ex:belongsToDomain ?domain.
}}
"""

q5 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?degree
WHERE {{
    ex:{best_teacher} ex:hasDegree ?degree.
    ex:{course} ex:requiresDegree ?degree.
}}
"""


In [243]:
skills = []
ans = graph.query(q1)
for t in ans:
    skills.append(str(t.skill).split("/")[-1])

relevant_experiences = []
ans = graph.query(q2)
for t in ans:
    role = str(t.role).split("/")[-1]
    organization = str(t.organization).split("/")[-1]
    relevant_experiences.append([role,int(t.years),organization])

research_work = []
ans = graph.query(q3)
for t in ans:
    research_work.append(str(t.research).split("/")[-1])

depart_domain = []
ans = graph.query(q4)
for t in ans:
    depart_domain.append(str(t.domain).split("/")[-1])

degrees = []
ans = graph.query(q5)
for t in ans:
    degrees.append(str(t.degree).split("/")[-1])


In [244]:
print("-----------------------------------------------------------------")
print(f"{best_teacher} is chosen as best_fit for {course} becuase of: ")
print("-----------------------------------------------------------------")

if depart_domain:
    print(f"\nThe {course} belongs to the following domain and the {best_teacher} is linked to the similar domains")
    for dom in depart_domain:
        print(dom)

if skills:
    print(f"\nFollowing Skills {best_teacher} has relevant to: ")
    for skill in skills:
        print(skill)

if degrees:
    print(f"{best_teacher} has underlying degrees relevant to the {course}")
    for degree in degrees:
        print(degree)

if research_work:
    print(f"\n{best_teacher} has various research contributions which shares relation to the {course}")
    for work in research_work:
        print(work)

if relevant_experiences:
    print(f"\nThe Relevant Expereince regarding {course} in following organizations: ")
    for exp in relevant_experiences:
        print("Job Role: ",exp[0],"\n Years: ",exp[1], " Organization: ",exp[2])





-----------------------------------------------------------------
Mr_Kashif is chosen as best_fit for IS_Audit_And_Control becuase of: 
-----------------------------------------------------------------

The IS_Audit_And_Control belongs to the following domain and the Mr_Kashif is linked to the similar domains
CoreComputing

Following Skills Mr_Kashif has relevant to: 
AuditProcess
InformationSecurity
AuditAndCompliance
ISCompliance
ISStrategyPlanning
SystemDevelopmentManagement
SecurityManagement
OperationsManagement
QualityAssurance
ComputerizedAuditSoftware
PerformanceMeasurementTools
DisasterRecoveryPlanning
BusinessContinuityPlanning
Mr_Kashif has underlying degrees relevant to the IS_Audit_And_Control
MastersInInformationSystems
BachelorsInComputerScience

Mr_Kashif has various research contributions which shares relation to the IS_Audit_And_Control
Security_issues_in_ERP
Risk_Planning_in_Core_Banking_Applications
Change_Management_in_Banking_Sectors
IT_Governance_in_Corporate_Sec

In [250]:
# RUNNABLE
import tkinter as tk
from tkinter import messagebox
from rdflib import Graph, Namespace, RDF

ontology_file = 'Latest Ontology.ttl'
graph = Graph()
graph.parse(ontology_file, format='ttl')
ex = Namespace('http://example.org#')


course = "IS_Audit_And_Control"

q1 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?skill) AS ?matchedSkillCount) 
WHERE {{
    ?teacher ex:hasSkill ?skill.
    ex:{course} ex:requiresSkill ?skill.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedSkillCount)"""


q2 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?role) AS ?matchedCount)  
WHERE {{
    ?teacher ex:hasExperience ?exp.
    ?exp ex:hasRole ?role.
    ?role ex:roleRelevantTo ex:{course}.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedCount)
"""


q3 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (SUM(?years) AS ?totalYears) (COUNT(?role) AS ?matchedCount)  
WHERE {{
    ?teacher ex:hasExperience ?exp.
    ?exp ex:hasRole ?role.
    ?role ex:roleRelevantTo ex:{course}.
    ?exp ex:yearsOfExperience ?years.
}}
GROUP BY ?teacher
ORDER BY DESC(?totalYears)

"""

q4 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?research) AS ?matchedCount)  
WHERE {{
    ?teacher ex:doneResearch ?research.
    ?research ex:researchRelatedTo ex:{course}.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedCount)
"""

q5 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?domain) AS ?matchedCount)  
WHERE {{
    ?teacher ex:relatedToDomain ?domain.
    ex:{course} ex:belongsToDomain ?domain.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedCount)

"""

q6 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?teacher (COUNT(?degree) AS ?matchedCount)  
WHERE {{
    ?teacher ex:hasDegree ?degree.
    ex:{course} ex:requiresDegree ?degree.
}}
GROUP BY ?teacher
ORDER BY DESC(?matchedCount)
"""

import pandas as pd


scores = {}
results = graph.query(q1)
for result in results:
    teacher = str(result.teacher).split("/")[-1]
    matched_skills = int(result.matchedSkillCount)
    if teacher not in scores:
        scores[teacher] = {"matchedSkills": 0, "jobExperience": 0, "weightedExperience": 0, "researchExperience": 0, "domainMatching": 0, "degreeMatching": 0}
    scores[teacher]["matchedSkills"] = matched_skills


results = graph.query(q2)
for result in results:
    teacher = str(result.teacher).split("/")[-1]
    matched_exp = int(result.matchedCount)
    if teacher not in scores:
        scores[teacher] = {"matchedSkills": 0, "jobExperience": 0, "weightedExperience": 0, "researchExperience": 0, "domainMatching": 0, "degreeMatching": 0}
    scores[teacher]["jobExperience"] = matched_exp

results = graph.query(q3)
for result in results:
    teacher = str(result.teacher).split("/")[-1]
    total_years = int(result.totalYears)
    if teacher not in scores:
        scores[teacher] = {"matchedSkills": 0, "jobExperience": 0, "weightedExperience": 0, "researchExperience": 0, "domainMatching": 0, "degreeMatching": 0}
    scores[teacher]["weightedExperience"] = total_years

results = graph.query(q4)
for result in results:
    teacher = str(result.teacher).split("/")[-1]
    matched_research = int(result.matchedCount)
    if teacher not in scores:
        scores[teacher] = {"matchedSkills": 0, "jobExperience": 0, "weightedExperience": 0, "researchExperience": 0, "domainMatching": 0, "degreeMatching": 0}
    scores[teacher]["researchExperience"] = matched_research

results = graph.query(q5) 
for result in results:
    teacher = str(result.teacher).split("/")[-1]
    matched_domains = int(result.matchedCount)
    if teacher not in scores:
        scores[teacher] = {"matchedSkills": 0, "jobExperience": 0, "weightedExperience": 0, "researchExperience": 0, "domainMatching": 0, "degreeMatching": 0}
    scores[teacher]["domainMatching"] = matched_domains

results = graph.query(q6)
for result in results:
    teacher = str(result.teacher).split("/")[-1]
    matched_degrees = int(result.matchedCount)
    if teacher not in scores:
        scores[teacher] = {"matchedSkills": 0, "jobExperience": 0, "weightedExperience": 0, "researchExperience": 0, "domainMatching": 0, "degreeMatching": 0}
    scores[teacher]["degreeMatching"] = matched_degrees


df = pd.DataFrame.from_dict(scores, orient="index")
df.index.name = "Teacher"
df.reset_index(inplace=True)
df['Score'] = df["matchedSkills"] + df["jobExperience"] + df["weightedExperience"] + df["researchExperience"] + df["domainMatching"] + df["degreeMatching"]
best_teacher = df[ df["Score"] == df["Score"].max()]["Teacher"].values[0]


q1_1 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?skill
WHERE {{
    ex:{best_teacher} ex:hasSkill ?skill.
    ex:{course} ex:requiresSkill ?skill.
}} """


q2_1 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?exp ?role ?years ?organization
WHERE {{
    ex:{best_teacher} ex:hasExperience ?exp.
    ?exp ex:hasRole ?role.
    ?role ex:roleRelevantTo ex:{course}.
    ?exp ex:yearsOfExperience ?years. 
    ?exp ex:organization ?organization.
}}"""


q3_1 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?research
WHERE {{
    ex:{best_teacher} ex:doneResearch ?research.
    ?research ex:researchRelatedTo ex:{course}.
}}
"""

q4_1 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?domain
WHERE {{
    ex:{best_teacher} ex:relatedToDomain ?domain.
    ex:{course} ex:belongsToDomain ?domain.
}}
"""

q5_1 = f"""PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ex: <http://example.org/ontology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?degree
WHERE {{
    ex:{best_teacher} ex:hasDegree ?degree.
    ex:{course} ex:requiresDegree ?degree.
}}
"""

skills = []
ans = graph.query(q1_1)
for t in ans:
    skills.append(str(t.skill).split("/")[-1])

relevant_experiences = []
ans = graph.query(q2_1)
for t in ans:
    role = str(t.role).split("/")[-1]
    organization = str(t.organization).split("/")[-1]
    relevant_experiences.append([role,int(t.years),organization])

research_work = []
ans = graph.query(q3_1)
for t in ans:
    research_work.append(str(t.research).split("/")[-1])

depart_domain = []
ans = graph.query(q4_1)
for t in ans:
    depart_domain.append(str(t.domain).split("/")[-1])

degrees = []
ans = graph.query(q5_1)
for t in ans:
    degrees.append(str(t.degree).split("/")[-1])


print("-----------------------------------------------------------------")
print(f"{best_teacher} is chosen as best_fit for {course} becuase of:   ")
print("-----------------------------------------------------------------")

if depart_domain:
    print(f"\nThe {course} belongs to the following domain and the {best_teacher} is linked to the similar domains")
    for dom in depart_domain:
        print(dom)

if skills:
    print(f"\nFollowing Skills {best_teacher} has relevant to: ")
    for skill in skills:
        print(skill)

if degrees:
    print(f"{best_teacher} has underlying degrees relevant to the {course}")
    for degree in degrees:
        print(degree)

if research_work:
    print(f"\n{best_teacher} has various research contributions which shares relation to the {course}")
    for work in research_work:
        print(work)

if relevant_experiences:
    print(f"\nThe Relevant Expereince regarding {course} in following organizations: ")
    for exp in relevant_experiences:
        print("Job Role: ",exp[0],"\n Years: ",exp[1], " Organization: ",exp[2])

-----------------------------------------------------------------
Mr_Kashif is chosen as best_fit for IS_Audit_And_Control becuase of: 
-----------------------------------------------------------------

The IS_Audit_And_Control belongs to the following domain and the Mr_Kashif is linked to the similar domains
CoreComputing

Following Skills Mr_Kashif has relevant to: 
AuditProcess
InformationSecurity
AuditAndCompliance
ISCompliance
ISStrategyPlanning
SystemDevelopmentManagement
SecurityManagement
OperationsManagement
QualityAssurance
ComputerizedAuditSoftware
PerformanceMeasurementTools
DisasterRecoveryPlanning
BusinessContinuityPlanning
Mr_Kashif has underlying degrees relevant to the IS_Audit_And_Control
MastersInInformationSystems
BachelorsInComputerScience

Mr_Kashif has various research contributions which shares relation to the IS_Audit_And_Control
Security_issues_in_ERP
Risk_Planning_in_Core_Banking_Applications
Change_Management_in_Banking_Sectors
IT_Governance_in_Corporate_Sec